<font color='darkorange'> Unless otherwise noted, **this notebook will not be reviewed or autograded.**</font> You are welcome to use it for scratchwork, but **only the files listed in the exercises will be checked.**

---

# Exercises

For these exercises, add your functions to the *apputil\.py* file and *app\.py* file as instructed. *These exercises use the same [Titanic dataset](https://www.kaggle.com/competitions/titanic/data) as the lab.*


## Exercise 1: Survival Patterns


For this exercise you will analyze survival patterns on the Titanic by looking at passenger class, sex, and age group. Name the function `survival_demographics()`.

1. Create a new column in the Titanic dataset that classifies passengers into age categories (i.e., a pandas `category` series). The categories should be:
    - Child (up to 12)
    - Teen (13–19)
    - Adult (20–59)
    - Senior (60+)  
  
	Hint: The `pd.cut()` function might come in handy here.

2. Group the passengers by class, sex, and age group.  

3. For each group, calculate:  
    - The total number of passengers, `n_passengers`
    - The number of survivors, `n_survivors`
    - The survival rate, `survival_rate`

4. Return a table that includes the results for *all* combinations of class, sex, and age group.  

5. Order the results so they are easy to interpret.  

6. Come up with a clear question that your results table makes you curious about (e.g., “Did women in first class have a higher survival rate than men in other classes?”). Write this question in your `app.py` file above the call to your visualization function, using `st.write("Your Question Here")`.
   
7. Create a Plotly visualization in a function named `visualize_demographic()` that directly addresses your question by returning a Plotly figure (e.g., `fig = px. ...`). You are free to choose the chart type that you think best communicates the findings. Be creative — try different approaches, compare them, and ensure that your chart clearly answers the question you posed.


import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

In [24]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
    age_bins = [0, 12, 19, 59, float('inf')]
    age_catagory = ['Child', 'Teen', 'Adult', 'Senior']
    df['age_group'] = pd.cut(df['Age'], bins=age_bins, labels=age_catagory, right=True)
    df['age_group'] = df['age_group'].astype('category')

    # 2. Group the passengers by class, sex, and age group
    grouped = df.groupby(['Pclass', 'Sex', 'age_group'], observed=True)

    # Step 3: For each group, calculate n_passengers,n_survivors, and survival_rate
    data_summary = grouped.agg(
        n_passengers=('PassengerId', 'count'),
        n_survivors=('Survived', 'sum')
    ).reset_index()

    data_summary['survival_rate'] = data_summary['n_survivors'] / data_summary['n_passengers']

NameError: name 'df' is not defined

In [1]:
def survival_demographics(df):
    # 1. Create a new column in the Titanic dataset that classifies passengers into age categories
    age_bins = [0, 12, 19, 59, float('inf')]
    age_catagory = ['Child', 'Teen', 'Adult', 'Senior']
    df['age_group'] = pd.cut(df['Age'], bins=age_bins, labels=age_catagory, right=True)
    df['age_group'] = df['age_group'].astype('category')

    # 2. Group the passengers by class, sex, and age group
    grouped = df.groupby(['Pclass', 'Sex', 'age_group'], observed=True)

    # 3. For each group, calculate n_passengers,n_survivors, and survival_rate
    #4. Return a table that includes the results for all combinations of class, sex, and age group.
    data_summary = grouped.agg(
        n_passengers=('PassengerId', 'count'),
        n_survivors=('Survived', 'sum')
    ).reset_index()

    data_summary['survival_rate'] = data_summary['n_survivors'] / data_summary['n_passengers']

    # 5. Order the results so they are easy to interpret
    data_summary = data_summary.sort_values(by=['Pclass', 'Sex', 'age_group'])

    return data_summary

In [6]:
def visualize_demographic(summary_df):
    # Filter to highlight adult women in first class
    highlight = (
        (summary_df['Pclass'] == 1) &
        (summary_df['Sex'] == 'female') &
        (summary_df['age_group'] == 'Adult')
    )

    # Add a highlight column for color emphasis
    summary_df['highlight'] = highlight.map({True: 'Highlighted', False: 'Other'})

    # Create bar chart
    fig = px.bar(
        summary_df,
        x='survival_rate',
        y='age_group',
        color='highlight',
        facet_row='Sex',
        facet_col='Pclass',
        orientation='h',
        title='Survival Rate by Class, Sex, and Age Group',
        labels={'survival_rate': 'Survival Rate', 'age_group': 'Age Group'},
        color_discrete_map={'Highlighted': 'crimson', 'Other': 'lightgray'}
    )

    fig.update_layout(showlegend=False)
    return fig

In [2]:
import pandas as pd
import streamlit as st

In [9]:
import plotly.express as px

In [10]:
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

st.write(
'''
# Titanic Visualization 1

'''
"Did women in first class have a higher survival rate than men in other classes?"
)
# Generate and display the figure
summary_df = survival_demographics(df)
fig1 = visualize_demographic(summary_df)
st.plotly_chart(fig1, use_container_width=True)

2025-09-29 20:21:45.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:21:45.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:21:45.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-09-29 20:21:45.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:21:45.856 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:21:45.857 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:21:45.858 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:21:45.859 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

import streamlit as st

In [31]:
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')

st.write(
'''
# Titanic Visualization 1

'''
"Did women in first class have a higher survival rate than men in other classes?"
)
# Generate and display the figure
fig1 = visualize_demographic()
st.plotly_chart(fig1, use_container_width=True)

2025-09-29 20:06:00.166 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2025-09-29 20:06:00.302 
  command:

    streamlit run /root/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-29 20:06:00.302 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-29 20:06:00.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


TypeError: visualize_demographic() missing 1 required positional argument: 'summary_df'

In [ ]:
data_summary

,Pclass,Sex,age_group,n_passengers,n_survivors,survival_rate
0,1,female,Child,1,0,0.000000
1,1,female,Teen,13,13,1.000000
2,1,female,Adult,68,66,0.970588
3,1,female,Senior,3,3,1.000000
4,1,male,Child,3,3,1.000000
5,1,male,Teen,4,1,0.250000
6,1,male,Adult,80,34,0.425000
7,1,male,Senior,14,2,0.142857
8,2,female,Child,8,8,1.000000
9,2,female,Teen,8,8,1.000000


    age_bins = [0, 12, 19, 59, float('inf')]
    age_catagory = ['Child', 'Teen', 'Adult', 'Senior']
    df['age_group'] = pd.cut(df['Age'], bins=age_bins, labels=age_catagory, right=True)
    df['age_group'] = df['age_group'].astype('category')

    # 2. Group the passengers by class, sex, and age group
    grouped = df.groupby(['Pclass', 'Sex', 'age_group'], observed=True)

    # Step 3: For each group, calculate n_passengers,n_survivors, and survival_rate
    data_summary = grouped.agg(
        n_passengers=('PassengerId', 'count'),
        n_survivors=('Survived', 'sum')
    ).reset_index()

    data_summary['survival_rate'] = data_summary['n_survivors'] / data_summary['n_passengers']

In [ ]:
age_bin = [0, 12, 19, 59, float('inf')]
age_catagory = ['Child', 'Teen', 'Adult', 'Senior']

In [ ]:
df['Passenger_catagory'] = pd.cut(df['Age'], bins=age_bin, labels=age_catagory, right=True)
df['Passenger_catagory'] = df['Passenger_catagory'].astype('category')

In [ ]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Passenger_catagory'],
      dtype='object')

In [ ]:
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Passenger_catagory
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult


In [ ]:
 grouped_df = df.groupby(['Pclass', 'Sex', 'Passenger_catagory'], observed=True)

In [ ]:
grouped_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Passenger_catagory
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Adult
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Adult
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Adult
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Adult
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,685,0,2,"Brown, Mr. Thomas William Solomon",male,60.0,1,1,29750,39.0000,NaN,S,Senior
748,749,0,1,"Marvin, Mr. Daniel Warner",male,19.0,1,0,113773,53.1000,D30,S,Teen
802,803,1,1,"Carter, Master. William Thornton II",male,11.0,1,2,113760,120.0000,B96 B98,S,Child
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0000,B28,NaN,Senior


## Exercise 2: Family Size and Wealth

Using the Titanic dataset, write a function named `family_groups()` to explore the relationship between family size, passenger class, and ticket fare.  

1. Create a new column in the Titanic dataset that represents the total family size for each passenger, `family_size`. Family size is defined as the number of siblings/spouses aboard plus the number of parents/children aboard, plus the passenger themselves.

2. Group the passengers by family size and passenger class. For each group, calculate:  
   - The total number of passengers, `n_passengers`
   - The average ticket fare, `avg_fare`
   - The minimum and maximum ticket fares (to capture variation in wealth), `min_fare` and `max_fare`

3. Return a table with these results, sorted so that the values are clear and easy to interpret (for example, by class and then family size).

4. Write a function called `last_names()` that extracts the last name of each passenger from the `Name` column, and returns the count for each last name (i.e., a pandas series with last name as index, and count as value). Does this result agree with that of the data table above? Share your findings in your app using `st.write`.

5. Just like you did in Exercise 1, come up with a clear question that your results makes you curious about. Write this question in your app.py file above the call to your visualization function. Then, create a Plotly visualization in a function named `visualize_families()` that directly addresses your question. As in Exercise 1 you are free to choose the chart type that you think best communicates the findings.

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/leontoddjohnson/datasets/main/data/titanic.csv')
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
def family_groups(df):
# 1. Create a new column in the Titanic dataset that calculates the family size for each passenger
    df['family_size']=df['SibSp'] + df['Parch'] + 1
# 2. Group the passengers by family size and class
    groupe_fam_class=df.groupby(['family_size', 'Pclass'],observed=True)
# 3. For each group, calculate n_passengers, avg_fare, min_fare, and max_fare
    summary=groupe_fam_class.agg(
    n_passengers=('PassengerId', 'count'),avg_fare=('Fare', 'mean'),min_fare=('Fare', 'min'),max_fare=('Fare', 'max')).reset_index()  
#4. Return a table that sort the result for easily interpreting first by class and then Family Size
    summary= summary.sort_values(by=['Pclass', 'family_size'])
    return summary




    
        
       
        

In [1]:
def last_names(df):
    df['last_names'] = df['Name'].str.extract(r',\s*([^.]*)\.')
    last_names_count=df['last_names'].value_counts()
    return last_names_count



In [21]:
 df['family_size']=df['SibSp'] + df['Parch'] + 1
 df['family_size'].tail()

886    1
887    1
888    4
889    1
890    1
Name: family_size, dtype: int64

In [25]:
groupe_fam_class=df.groupby(['family_size', 'Pclass'],observed=True)
groupe_fam_class.head()



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,family_size
0,1,0,3,"Braund, Mr. Owen Harris",male,22.00,1,0,A/5 21171,7.2500,NaN,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.00,1,0,PC 17599,71.2833,C85,C,2
2,3,1,3,"Heikkinen, Miss. Laina",female,26.00,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.00,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.00,0,0,373450,8.0500,NaN,S,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,645,1,3,"Baclini, Miss. Eugenie",female,0.75,2,1,2666,19.2583,NaN,C,4
678,679,0,3,"Goodwin, Mrs. Frederick (Augusta Tyler)",female,43.00,1,6,CA 2144,46.9000,NaN,S,8
742,743,1,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21.00,2,2,PC 17608,262.3750,B57 B59 B63 B66,C,5
774,775,1,2,"Hocking, Mrs. Elizabeth (Eliza Needs)",female,54.00,1,3,29105,23.0000,NaN,S,5


In [27]:
summary=groupe_fam_class.agg(n_passengers=('PassengerId', 'count'),avg_fare=('Fare', 'mean'),min_fare=('Fare', 'min'),max_fare=('Fare', 'max'))    
summary.head

<bound method NDFrame.head of                     n_passengers    avg_fare  min_fare  max_fare
family_size Pclass                                              
1           1                109   63.672514    0.0000  512.3292
            2                104   14.066106    0.0000   73.5000
            3                324    9.272052    0.0000   56.4958
2           1                 70   91.848039   29.7000  512.3292
            2                 34   24.682962   11.5000   33.0000
            3                 57   13.410023    6.4958   24.1500
3           1                 24   95.681075   26.2833  211.5000
            2                 31   31.693819   13.0000   73.5000
            3                 47   16.378277    7.2292   24.1500
4           1                  7  133.521429  120.0000  151.5500
            2                 13   36.575969   11.5000   65.0000
            3                  9   20.103689   15.8500   23.4500
5           1                  2  262.375000  262.3750  262.

In [33]:
summary= summary.sort_values(by=['Pclass', 'family_size'])
summary.tail()



,,n_passengers,avg_fare,min_fare,max_fare
family_size,Pclass,,,,
5,3,12,26.972233,21.075,34.3750
6,3,17,32.420588,27.900,39.6875
7,3,12,29.366667,7.925,31.3875
8,3,6,46.900000,46.900,46.9000
11,3,7,69.550000,69.550,69.5500


## Bonus Question

Add a new column, `older_passenger`, to the Titanic dataset that indicates whether each passenger’s age is above the median age for *their* passenger class. So, suppose row $x$ is in passenger class 2. Then, a value of `True` at row $x$ would indicate that passenger older than 50% of class 2 passengers, and `False` would indicate that they younger.

- You should use pandas functions to accomplish this.
- The new column should contain Boolean values (True if the age is above the median, False if less than or equal to).
- Return the updated table in the function `determine_age_division()`

Once you’ve created this column, consider how this age division relates to your analysis above. Try to visualize this analysis in Plotly using the function name `visualize_age_division()`.